In [3]:
from fastai import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 
import pathlib
from pathlib import Path
from datetime import datetime
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.collect_env import *
import cv2
import torch
%matplotlib inline

In [4]:
mpl.rcParams['figure.figsize'] = [15, 15] # Size in Inches
show_install()



```text
=== Software === 
python        : 3.7.3
fastai        : 1.0.57
fastprogress  : 0.1.21
torch         : 1.2.0
nvidia driver : 455.45
torch cuda    : 10.0.130 / is available
torch cudnn   : 7600 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : 16125MB | Quadro RTX 5000

=== Environment === 
platform      : Linux-5.4.0-62-generic-x86_64-with-debian-buster-sid
distro        : #70~18.04.1-Ubuntu SMP Tue Jan 12 17:18:00 UTC 2021
conda env     : pythongis
python        : /home/sandeep/anaconda3/envs/pythongis/bin/python
sys.path      : /home/sandeep/GoogleDrive/GTech/CS7643_DeepLearning/Project/building_footprints_cs7643
/home/sandeep/anaconda3/envs/pythongis/lib/python37.zip
/home/sandeep/anaconda3/envs/pythongis/lib/python3.7
/home/sandeep/anaconda3/envs/pythongis/lib/python3.7/lib-dynload

/home/sandeep/.local/lib/python3.7/site-packages
/home/sandeep/anaconda3/envs/pythongis/lib/python3.7/site-packages
/home/sandeep/anaconda3/envs/pythongis/lib/

## Download Data

We are using INRIA data for training building footprints. This can be further tuned to be refitted on futher target data. 

https://project.inria.fr/aerialimagelabeling/

This data is also available in our vm at:

<path-to-training-data>

In this bucket, we have :

- We have tiles as 512x512 images and masks in their respective folders. (this is used for training)



In [5]:
def subtile_images(src_imgs_path, src_img_file_ext, dst_folder, dst_img_file_ext, dst_imgX, dst_imgY, dst_img_format=''):
    ''' Method to split image files in smalled tile images. 
            src_imgs_path: folder path where all the image files to be splitted are present.
            src_img_file_ext: extension of the image files to be splitted
            dst_folder: folder path for saving splitted smaller images
            dst_img_file_ext: extension for splitted smaller images
            dst_imgX: width of tile
            dst_imgY: height of tile
        Note that all the masks will be saved as numpy array of dtype float with values as 0.0 and 1.10
            
    '''
    M = dst_imgX #512
    N = dst_imgY #512
    for img_file in Path(src_imgs_path).ls():
        if img_file.suffix != src_img_file_ext:
            continue
        im = cv2.imread(str(img_file))
        tiles = [im[x:x+M,y:y+N] for x in range(0,im.shape[0],M) for y in range(0,im.shape[1],N)]
        tiles = [t for t in tiles if  t.shape[:2]==(M,N)] #drop the tiles, which are not of size (M,N)
        for i,t in enumerate(tiles):
            if dst_img_format =='gray': #if needed save as grayscale images
                plt.imsave(str(Path(dst_folder)/f'{img_file.stem}_{i}{dst_img_file_ext}'), t, cmap=dst_img_format)
            elif len(t.shape)==2: #if it is mask, save as 0.0 or 1.0 values
                #cv2.imwrite(str(Path(dst_folder)/f'{img_file.stem}_{i}{dst_img_file_ext}'), t.astype('bool').astype(np.uint8))
                cv2.imwrite(Path(dst_folder)/f'{img_file.stem}_{i}{dst_img_file_ext}', t)
            else: #Else save as normal 3 or 4 channel image 
                cv2.imwrite(str(Path(dst_folder)/f'{img_file.stem}_{i}{dst_img_file_ext}'), t)

In [6]:
path_imgs = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/original_data/AerialImageDataset/train/images')
path_masks = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/original_data/AerialImageDataset/train/gt')
path_save_imgs_512 = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/training_512x512/images-512')
path_save_masks_512 = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/training_512x512/masks-512')

path_save_imgs_256 = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/training_256x256/images-256')
path_save_masks_256 = Path('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/training_256x256/masks-256')

### 512 x 512 tiles

In [13]:
# Split all the training images
# Total Time taken : 25 mins
st=datetime.now()
subtile_images(path_imgs, '.tif',
               path_save_imgs_512, '.png',
               512, 512)
print(f'Total Time taken : {datetime.now()-st}')

Total Time taken : 0:25:46.649695


In [98]:
st=datetime.now()
# Split all the training masks
subtile_images(path_masks, '.tif',
               path_save_masks_512, '.png',
               512, 512)
print(f'Total Time taken : {datetime.now()-st}')

Total Time taken : 0:00:51.877299


## 256 x 256 tiles

In [7]:
# Split all the training images
# Total Time taken : 25 mins
st=datetime.now()
subtile_images(path_imgs, '.tif',
               path_save_imgs_256, '.png',
               256, 256)
print(f'Total Time taken : {datetime.now()-st}')

Total Time taken : 0:04:33.458065


In [9]:
path_save_masks_256

PosixPath('/home/sandeep/GoogleDrive/beans-home/mount_shared_partition/image_data/data_building_footprint/training_256x256/masks-256')

In [10]:
st=datetime.now()
# Split all the training masks
subtile_images(path_masks, '.tif',
               path_save_masks_256, '.png',
               256, 256)
print(f'Total Time taken : {datetime.now()-st}')

Total Time taken : 0:00:57.178332
